In [1]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)
import numpy as np
import cs_vqe as c
import itertools
import qubit_conversion as q_conv

In [2]:
import ast
import matplotlib
import matplotlib.pyplot as plt

f = open("hamiltonians.txt","r")
hamiltonians = ast.literal_eval(f.read())
f.close()

for h in hamiltonians.keys():
    print(h, '# qubits:', hamiltonians[h][1])

H2-S1_STO-3G_singlet # qubits: 18
C1-O1_STO-3G_singlet # qubits: 16
H1-Cl1_STO-3G_singlet # qubits: 16
H1-Na1_STO-3G_singlet # qubits: 16
H2-Mg1_STO-3G_singlet # qubits: 17
H1-F1_3-21G_singlet # qubits: 18
H1-Li1_3-21G_singlet # qubits: 18
Be1_STO-3G_singlet # qubits: 5
H1-F1_STO-3G_singlet # qubits: 8
H1-Li1_STO-3G_singlet # qubits: 8
Ar1_STO-3G_singlet # qubits: 13
F2_STO-3G_singlet # qubits: 15
H1-O1_STO-3G_singlet # qubits: 8
H2-Be1_STO-3G_singlet # qubits: 9
H2-O1_STO-3G_singlet # qubits: 10
H2_3-21G_singlet # qubits: 5
H2_6-31G_singlet # qubits: 5
H3-N1_STO-3G_singlet # qubits: 13
H4-C1_STO-3G_singlet # qubits: 14
Mg1_STO-3G_singlet # qubits: 13
N2_STO-3G_singlet # qubits: 15
Ne1_STO-3G_singlet # qubits: 5
O2_STO-3G_singlet # qubits: 15
H1-Li1-O1_STO-3G_singlet # qubits: 18
H1-He1_STO-3G_singlet # qubits: 2
H3_STO-3G_singlet_1+ # qubits: 3
H1-He1_3-21G_singlet_1+ # qubits: 6
H3_3-21G_singlet_1+ # qubits: 9
H4-N1_STO-3G_singlet_1+ # qubits: 14


In [3]:
speciesname = 'H1-He1_3-21G_singlet_1+'

encoding = hamiltonians[speciesname][0] # in this dataset, always 'JW' for Jordan-Wigner, but leaves room for trying Bravyi-Kitaev as well
num_qubits = hamiltonians[speciesname][1] # number of qubits (all of these Hamiltonians have been tapered for molecular symmetries)
ham = hamiltonians[speciesname][2] # full Hamiltonian
ham_noncon = hamiltonians[speciesname][3] # noncontextual part of Hamiltonian, found by greedy DFS
true_gs = hamiltonians[speciesname][4] # ground state energy of full Hamiltonian (in Hartree)
gs_noncon = hamiltonians[speciesname][5] # list containing information about noncontextual ground state: zeroth entry is ground state energy of noncontextual part of Hamiltonian
#print('commuting generators:',model[0], '\n')
#print('anticommuting generators:',model[1], '\n')
#print('term reconstruction:',model[2], '\n')

print('number of qubits:', num_qubits)
print('true ground state energy:', true_gs)
print('noncontextual approximation to ground state energy:', gs_noncon[0])
print(ham_noncon)

number of qubits: 6
true ground state energy: -4.259188160411326
noncontextual approximation to ground state energy: -4.235679984712036
{'IZIZIZ': 1.137117313013182, 'ZIZIZI': 1.137117313013182, 'ZIIIII': -0.6757846691545434, 'IZIIII': -0.6757846691545433, 'IIIIZZ': 0.24918329066787195, 'IIIZII': -0.24459538421117766, 'IIZIII': -0.24459538421117766, 'ZZZZZZ': 0.2324217371047461, 'IZIZZZ': -0.23101530963627243, 'ZIZIZZ': -0.23101530963627243, 'IIIIII': -0.22683086324346313, 'ZZIIII': 0.17119154268004644, 'IZZIII': 0.14145232769859284, 'ZIIZII': 0.14145232769859284, 'IZIIZI': 0.1411974641913617, 'ZIIIIZ': 0.1411974641913617, 'IIIIIZ': 0.13839367136530806, 'IIIIZI': 0.13839367136530806, 'IIZZII': 0.13734855781563715, 'IIIZZI': 0.1365862143604441, 'IIZIIZ': 0.1365862143604441, 'IZZZIZ': -0.12223086443933409, 'ZIZZZI': -0.12223086443933409, 'ZZIZIZ': -0.11934196603424856, 'ZZZIZI': -0.11934196603424856, 'IXZXII': -0.0627599937687078, 'IYZYII': -0.0627599937687078, 'IZIZII': -0.0620181618623

In [4]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)
import numpy as np
import cs_vqe as c
import itertools
import qubit_conversion as q_conv

singlet_bool = True # Set general UCCSD or singlet UCCSD.

bond_len = 0.772#1.45
atom_1 = 'He'
atom_2 = 'H'
basis = '3-21g'
multiplicity = 1
charge = 1

coordinate_1 = (0.0, 0.0, 0.0)
coordinate_2 = (0.0, 0.0, bond_len)
geometry = [(atom_1, coordinate_1), (atom_2, coordinate_2)]

molecule_data = MolecularData(geometry, basis, multiplicity, charge, description='Test')
#molecule.load()

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule_data,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci)

#molecule.load()
#print(molecule)

ham = get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_q = jordan_wigner(ham)
print('Hamiltonian:', '\n', ham_q, '\n')

scf = True      # Hartree-Fock.
mp2 = True      # Moller-Plesset 2.
cisd = True     # Configuration interaction singles and doubles.
ccsd = True     # Coupled cluster singles and doubles.
fci = True      # Full configuration interaction.

calculated_molecule = run_pyscf(molecule_data, scf, mp2, cisd, ccsd, fci)

if ccsd:
    ccsd_single_amps = calculated_molecule.ccsd_single_amps
    ccsd_double_amps = calculated_molecule.ccsd_double_amps

num_electrons = calculated_molecule.n_electrons
num_qubits = 2*calculated_molecule.n_orbitals

if singlet_bool:
    # Get singlet UCCSD generator.
    packed_amps = uccsd_singlet_get_packed_amplitudes(ccsd_single_amps,  ccsd_double_amps, num_qubits, num_electrons)
    ucc_sing = uccsd_singlet_generator(packed_amps, num_qubits, num_electrons)
    #print(ucc_sing)

else:
    # Get general UCCSD operator.
    ucc_op = uccsd_generator(ccsd_single_amps, ccsd_double_amps)
    #print(ucc_op)
    
ucc_q = jordan_wigner(ucc_sing)
print('UCCSD ansatz:', '\n', ucc_q)

Hamiltonian: 
 (1.1324434021069345+0j) [] +
(-0.025810254414816238+0j) [X0 X1 Y2 Y3] +
(-0.012799934786436543+0j) [X0 X1 Y2 Z3 Z4 Y5] +
(-0.025159645839898416+0j) [X0 X1 Y2 Z3 Z4 Z5 Z6 Y7] +
(-0.012799934786436542+0j) [X0 X1 X3 X4] +
(-0.025159645839898416+0j) [X0 X1 X3 Z4 Z5 X6] +
(-0.017114528821774402+0j) [X0 X1 Y4 Y5] +
(-0.0009891049103619052+0j) [X0 X1 Y4 Z5 Z6 Y7] +
(-0.0009891049103619052+0j) [X0 X1 X5 X6] +
(-0.05469856574684862+0j) [X0 X1 Y6 Y7] +
(0.025810254414816238+0j) [X0 Y1 Y2 X3] +
(0.012799934786436543+0j) [X0 Y1 Y2 Z3 Z4 X5] +
(0.025159645839898416+0j) [X0 Y1 Y2 Z3 Z4 Z5 Z6 X7] +
(-0.012799934786436542+0j) [X0 Y1 Y3 X4] +
(-0.025159645839898416+0j) [X0 Y1 Y3 Z4 Z5 X6] +
(0.017114528821774402+0j) [X0 Y1 Y4 X5] +
(0.0009891049103619052+0j) [X0 Y1 Y4 Z5 Z6 X7] +
(-0.0009891049103619052+0j) [X0 Y1 Y5 X6] +
(0.05469856574684862+0j) [X0 Y1 Y6 X7] +
(0.002577905179415873+0j) [X0 Z1 X2] +
(0.004766148105680716+0j) [X0 Z1 X2 X3 Z4 X5] +
(-0.009264702923308664+0j) [X0 Z1 X2 X3

UCCSD ansatz: 
 0.006478936130102838j [X0 X1 X2 Y3] +
0.006478936130102838j [X0 X1 Y2 X3] +
0.0020562744029710704j [X0 X1 X4 Y5] +
0.0020562744029710704j [X0 X1 Y4 X5] +
0.005083283191416048j [X0 X1 X6 Y7] +
0.005083283191416048j [X0 X1 Y6 X7] +
-0.006478936130102838j [X0 Y1 X2 X3] +
0.006478936130102838j [X0 Y1 Y2 Y3] +
-0.0020562744029710704j [X0 Y1 X4 X5] +
0.0020562744029710704j [X0 Y1 Y4 Y5] +
-0.005083283191416048j [X0 Y1 X6 X7] +
0.005083283191416048j [X0 Y1 Y6 Y7] +
-0.006360390047729467j [X0 Z1 Y2] +
-0.002645411873769119j [X0 Z1 Z2 Z3 Y4] +
0.0009221551709274561j [X0 Z1 Z2 Z3 Z4 Z5 Y6] +
-0.006478936130102838j [Y0 X1 X2 X3] +
0.006478936130102838j [Y0 X1 Y2 Y3] +
-0.0020562744029710704j [Y0 X1 X4 X5] +
0.0020562744029710704j [Y0 X1 Y4 Y5] +
-0.005083283191416048j [Y0 X1 X6 X7] +
0.005083283191416048j [Y0 X1 Y6 Y7] +
-0.006478936130102838j [Y0 Y1 X2 Y3] +
-0.006478936130102838j [Y0 Y1 Y2 X3] +
-0.0020562744029710704j [Y0 Y1 X4 Y5] +
-0.0020562744029710704j [Y0 Y1 Y4 X5] +
-0.0

In [5]:
ham = q_conv.QubitOperator_to_dict(ham_q, num_qubits)
anz_terms = list((q_conv.QubitOperator_to_dict(ucc_q, num_qubits)).keys())
terms_noncon = c.greedy_dfs(ham, 1, criterion='weight')[-1]
ham_noncon = {t:ham[t] for t in terms_noncon}
#ham_noncon
terms_context = list(ham.keys() - terms_noncon)
ham_context = {t:ham[t] for t in terms_context}
#ham_context
c.contextualQ_ham(ham_context)

True

In [6]:
from qiskit.aqua.algorithms import NumPyEigensolver

result = NumPyEigensolver(q_conv.dict_to_WeightedPauliOperator(ham)).run()
true_gs = np.real(result.eigenvalues)

print(true_gs)

/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/algorithms/eigen_solvers/eigen_solver.py:38: DeprecationWarning: The package qiskit.aqua.algorithms.eigen_solvers is deprecated. It was moved/refactored to qiskit.algorithms.eigen_solvers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  'qiskit-terra')


[-3.16676548]


/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')


In [7]:
model = c.quasi_model(ham_noncon)
fn_form = c.energy_function_form(ham_noncon, model)
gs_noncon = c.find_gs_noncon(ham_noncon)
gs_noncon_energy = gs_noncon[0]
ep_state = gs_noncon[1]
ham_context = {p:c for p,c in ham.items() if p not in ham_noncon}
c.csvqe_approximations_heuristic(ham, ham_noncon, num_qubits, true_gs)

[array([-3.16676548]),
 [-3.1428247492681756,
  -3.15187288186298,
  -3.151872881862978,
  -3.1557103882871265,
  -3.1591148519137593,
  -3.1591148519137526,
  -3.161949005650521,
  -3.166765477254425,
  -3.166765477254418],
 [array([0.02394073]),
  array([0.0148926]),
  array([0.0148926]),
  array([0.01105509]),
  array([0.00765063]),
  array([0.00765063]),
  array([0.00481647]),
  array([-1.11022302e-14]),
  array([-3.99680289e-15])],
 [6, 0, 2, 4, 3, 1, 5, 7]]

In [8]:
model[0:2], gs_noncon_energy, ep_state

((['ZIIIIIZI',
   'IZIIIIIZ',
   'IIZIIIII',
   'IIIZIIII',
   'IIIIZIII',
   'IIIIIZII',
   'IIIIIIIZ'],
  ['YZZZZZYZ', 'IIIIIIZI']),
 -3.1428247492681773,
 [[-1, -1, 1, -1, 1, 1, 1], [0.04017216410652921, 0.999192772807629]])

# The epistemic state defines a classical probability distribution:

In [9]:
def ontic_prob(ep_state, ontic_state):
    
    if ep_state[0] != ontic_state[0]:
        return 0
    
    else:
        prod = 1
        for index, r in enumerate(ep_state[1]):
            f = 1/2 * abs(r + ontic_state[1][index])
            prod *= f
        
        return prod    

def epistemic_dist(ep_state):
    size_G = len(ep_state[0])
    size_Ci = len(ep_state[1])
    size_R = size_G + size_Ci
    
    ep_prob = {}
    
    ontic_states = list(itertools.product([1, -1], repeat=size_R))
    
    for o in ontic_states:
        o_state = [list(o[0:size_G]), list(o[size_G:size_R])]
        o_prob = ontic_prob(ep_state, o_state)
        
        if o_prob != 0:
            ep_prob[o] = o_prob
    
    return ep_prob

In [10]:
epistemic_dist(ep_state)

{(-1, -1, 1, -1, 1, 1, 1, 1, 1): 0.519876168239361,
 (-1, -1, 1, -1, 1, 1, 1, 1, -1): 0.0002099138139035528,
 (-1, -1, 1, -1, 1, 1, 1, -1, 1): 0.47972021816445337,
 (-1, -1, 1, -1, 1, 1, 1, -1, -1): 0.0001936997822819619}

# Find a simultaneous eigenstate of the operators G_i and A:

In [26]:
r1 = ep_state[1][0]
r2 = ep_state[1][1]
t = np.arctan((r2 - 1) / r1)
t

-0.020091488476197236

In [27]:
def bin_to_int(bin_str):
    bits = [int(b) for b in bin_str]
    for index, b in enumerate(bits):
        bits[index] = b * 2 ** (len(bits)-index-1)
    return sum(bits)

psi = [0 for i in range(2**num_qubits)]
psi[bin_to_int('01010010')] = np.sin(t)
psi[bin_to_int('11010000')] = np.cos(t)
print(psi)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.020090136788621555, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9997981728348049, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [28]:
from openfermion.linalg import LinearQubitOperator

psi = np.array(psi)
ham_q = q_conv.dict_to_QubitOperator(ham, num_qubits)
ham_noncon_q = q_conv.dict_to_QubitOperator(ham_noncon, num_qubits)
ham_context_q = q_conv.dict_to_QubitOperator(ham_context, num_qubits)

#take expectation value algebraically
H = LinearQubitOperator(ham_q, num_qubits)
H_psi = H.matvec(psi)
expect = psi.dot(H_psi)

print('<H> w.r.t. simultaneous eigenstate:', expect)
print('Noncontextual approximation:', gs_noncon_energy)
print('Expectation values match?', abs(expect - gs_noncon_energy) < 10**-12) #sanity check - expectation values match? 

<H> w.r.t. simultaneous eigenstate: (-3.1386757795732274+0j)
Noncontextual approximation: -3.1428247492681773
Expectation values match? False


# Do we have any quantum corrections?

In [29]:
G = model[0]
terms_context = list(ham_context.keys())
G

['ZIIIIIZI',
 'IZIIIIIZ',
 'IIZIIIII',
 'IIIZIIII',
 'IIIIZIII',
 'IIIIIZII',
 'IIIIIIIZ']

In [30]:
q_corr_terms = []
for t in terms_context:
    commutes = []
    for g in G:
        if c.commute(t, g):
            commutes.append(g)
    print(t, 'commutes with the noncontextual generators:', commutes)
    if commutes == G:
        q_corr_terms.append(t)
        
q_corr_terms

YZYIIIII commutes with the noncontextual generators: ['IZIIIIIZ', 'IIIZIIII', 'IIIIZIII', 'IIIIIZII', 'IIIIIIIZ']
XZXIIIII commutes with the noncontextual generators: ['IZIIIIIZ', 'IIIZIIII', 'IIIIZIII', 'IIIIIZII', 'IIIIIIIZ']
YZZZYIII commutes with the noncontextual generators: ['IZIIIIIZ', 'IIZIIIII', 'IIIZIIII', 'IIIIIZII', 'IIIIIIIZ']
XZZZXIII commutes with the noncontextual generators: ['IZIIIIIZ', 'IIZIIIII', 'IIIZIIII', 'IIIIIZII', 'IIIIIIIZ']
IYZYIIII commutes with the noncontextual generators: ['ZIIIIIZI', 'IIZIIIII', 'IIIIZIII', 'IIIIIZII', 'IIIIIIIZ']
IXZXIIII commutes with the noncontextual generators: ['ZIIIIIZI', 'IIZIIIII', 'IIIIZIII', 'IIIIIZII', 'IIIIIIIZ']
IYZZZYII commutes with the noncontextual generators: ['ZIIIIIZI', 'IIZIIIII', 'IIIZIIII', 'IIIIZIII', 'IIIIIIIZ']
IXZZZXII commutes with the noncontextual generators: ['ZIIIIIZI', 'IIZIIIII', 'IIIZIIII', 'IIIIZIII', 'IIIIIIIZ']
IYZZZZZY commutes with the noncontextual generators: ['ZIIIIIZI', 'IZIIIIIZ', 'IIZIIIII'

[]

In [31]:
#psi2 = [0 for i in range(2**num_qubits)]
#psi2[bin_to_int('000011')] = np.sin(t)
#psi2[bin_to_int('010111')] = -np.cos(t)

In [32]:
#from qiskit import QuantumCircuit
#from qiskit.extensions import Initialize
#from qiskit.circuit import Parameter

#anz = QuantumCircuit(num_qubits) # We are redefining qc
#anz.initialize(psi)
#anz.ry(Parameter('x'), 3)
#anz.initialize(psi2)
#anz.rx(Parameter('y'), 3)

In [33]:
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.algorithms import VQE
from qiskit.quantum_info.operators.symplectic.pauli import Pauli
from qiskit.opflow.primitive_ops import PauliOp
from qiskit import Aer

ham_qiskit = sum([PauliOp(Pauli(k), ham[k]) for k in ham.keys()])

seed = 50
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)
slsqp = SLSQP(maxiter=1000)

vqe = VQE(anz, optimizer=slsqp, quantum_instance=qi)
vqe_run    = vqe.compute_minimum_eigenvalue(operator=ham_qiskit)
vqe_result = vqe_run.optimal_value# + shift

print('VQE:', vqe_result, '|', 'Noncontextual Ground State:', gs_noncon_energy, 'True Ground State:', true_gs)

NameError: name 'anz' is not defined

In [34]:
from qiskit.aqua.algorithms import NumPyEigensolver

order = [0,1,2,3,4,5,6,7]
print(c.contextual_subspace_approximations(ham,model,fn_form,ep_state,order))

result = NumPyEigensolver(q_conv.dict_to_WeightedPauliOperator(ham)).run()
exact_energy = np.real(result.eigenvalues)

print(exact_energy)

[-3.1428247492681756, -3.1428247492681756, -3.142824749268174, -3.143939975054507, -3.1459507148541945, -3.148913349475536, -3.154301104750833, -3.1667654772544216, -3.166765477254406]
[-3.16676548]


In [35]:
rotations = (c.diagonalize_epistemic(model,fn_form,ep_state))[0]
rotations

[['pi/2', 'YIIIIIII'],
 ['pi/2', 'YIIIIIZI'],
 ['pi/2', 'IYIIIIII'],
 ['pi/2', 'IYIIIIIZ']]

In [36]:
from copy import deepcopy

rot_psi = deepcopy(psi)

for r in rotations:
    r_op = LinearQubitOperator(QubitOperator('', 1/np.sqrt(2)) + q_conv.dict_to_QubitOperator({r[1]: -1/np.sqrt(2)*1j}, num_qubits), num_qubits)
    rot_psi = r_op.matvec(rot_psi)

In [37]:
from openfermion.linalg import LinearQubitOperator

psi = np.array(rot_psi)
rot_ham = c.get_reduced_hamiltonians(ham,model,fn_form,ep_state,order=[0,1,2,3,4,5,6,7])[8]
rot_ham_q = q_conv.dict_to_QubitOperator(rot_ham, num_qubits)

#take expectation value algebraically
H = LinearQubitOperator(rot_ham_q, num_qubits)
H_psi = H.matvec(rot_psi)
expect = psi.dot(H_psi)

print('<H> w.r.t. simultaneous eigenstate:', expect)
print('Noncontextual approximation:', gs_noncon_energy)
print('Expectation values match?', abs(expect - gs_noncon_energy) < 10**-14) #sanity check - expectation values match? 

<H> w.r.t. simultaneous eigenstate: (-3.1428247492681725+0j)
Noncontextual approximation: -3.1428247492681773
Expectation values match? True


In [ ]:
rot_psi.dot(H.matvec(rot_psi)), psi.dot(H.matvec(psi))

In [ ]:
num_qubits